# Advanved RAG (Part 2)

Continuation from [first part](https://github.com/jzamalloa1/langchain_learning/blob/main/advanced_rag.ipynb)

In [2]:
import pandas as pd
import numpy as np
import json, os, pprint
import matplotlib.pyplot as plt
import plotly.express as px
import random
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import JsonOutputToolsParser, JsonOutputKeyToolsParser
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_react_agent, Tool
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.tools import PythonREPLTool
from langchain import hub
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.callbacks import Callbacks
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.runnables import RunnablePassthrough

In [3]:
os.environ["OPENAI_API_KEY"] = ""

In [4]:
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.1, streaming=True)

## <center> Routing </center>

<p>
<img src="ILLUSTRATIONS/rag_diagram_routing.png" 
      width="65%" height="auto"
      style="display: block; margin: 0 auto" />

Illustration [reference](https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_10_and_11.ipynb)

In [10]:
from typing import Literal

class RouterQuery(BaseModel):
    """Route user query to most relevant data source"""

    datasource: Literal["python_docs", "web_broswer"] = Field(
        ...,
        description = "Given a user question choose the most appropriate datasource to answer it"
    )

In [11]:
structured_llm = llm.with_structured_output(RouterQuery)

/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [15]:
system_prompt = """
You are an expert router of user's questions. Based on the question asked, route it to
the appropriate data source
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}")
    ]
)

# Define router
router = prompt | structured_llm

In [17]:
query = """
how should I calcuate the mean of two vectors in numpy?
"""

router.invoke({"question":query})

RouterQuery(datasource='python_docs')

In [18]:
query = """
what is the weather in NYC today?
"""

router.invoke({"question":query})

RouterQuery(datasource='web_broswer')